In [1]:
import torch

import os
import json

from pos_sae.model import SparseAutoencoder
from pos_sae.compute_dead import get_freq_single_sae

from transformer_lens import HookedTransformer
from transformer_lens.utils import get_act_name

import plotly.express as px
import pandas as pd

from datasets import load_dataset
from torch.utils.data import DataLoader

/Users/slava/fun/pos_sae/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dir_path = "/Users/slava/fun/pos_sae/converted_checkpoints/final_sparse_autoencoder_gpt2-small_blocks.5.hook_resid_pre_24576"

model = SparseAutoencoder.load_from_pretrained(dir_path=dir_path)

model.to("mps")


Run name: 24576-L1-8e-05-LR-0.0004-Tokens-3.000e+08
ignored keys: ['hook_point_head_index', 'dataset_path', 'is_dataset_tokenized', 'use_cached_activations', 'cached_activations_path', 'n_batches_in_buffer', 'total_training_tokens', 'store_batch_size', 'seed', 'b_dec_init_method', 'use_ghost_grads', 'feature_sampling_window', 'dead_feature_window', 'wandb_entity', 'n_checkpoints']


SparseAutoencoder(
  (hook_sae_in): HookPoint()
  (hook_hidden_pre): HookPoint()
  (hook_hidden_post): HookPoint()
  (hook_sae_out): HookPoint()
)

In [3]:

gpt2 = HookedTransformer.from_pretrained("gpt2-small")


Loaded pretrained model gpt2-small into HookedTransformer


In [27]:
freqs = get_freq_single_sae(model, gpt2, n_batches=1000).cpu()
log_freq = torch.log10(freqs + 1e-6)

/Users/slava/fun/pos_sae/.venv/lib/python3.9/site-packages/datasets/load.py:1461: FutureWarning:

The repository for Skylion007/openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Skylion007/openwebtext
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.

100%|██████████| 1000/1000 [01:03<00:00, 15.65it/s]


In [28]:
print(freqs.shape) # (24576,)

torch.Size([24576])


In [29]:
print(freqs)
print(log_freq)

tensor([0.0003, 0.0003, 0.0021,  ..., 0.1234, 0.0008, 0.0002])
tensor([-3.4625, -3.4775, -2.6838,  ..., -0.9087, -3.1023, -3.6985])


In [30]:
# print freqs stats

print("Frequency Statistics:")
print(f"  Mean: {freqs.mean():.4f}")
print(f"  Std: {freqs.std():.4f}")
print(f"  Maximum: {freqs.max():.4f}")
print(f"  Minimum: {freqs.min():.4f}")


fig = px.histogram(x=log_freq, nbins=100)
fig.update_layout(
    title='SAE Feature Frequency Histogram',
    xaxis_title='Frequency (Log Scale)',
    yaxis_title='Count'
)
fig.show()

Frequency Statistics:
  Mean: 0.0019
  Std: 0.0080
  Maximum: 0.4001
  Minimum: 0.0000
